# API vs Web Craping

- Extracting informaiton from websites can be done via scraping or by working with the site API if there is one 
    - working with APIs is preferable 
    - Comparison of Web Scraping vs. API for Hacker News
    
## RAPTOR 
Raptor means: Review - Access - Parse - Transorm -stORe.

| |Web Server | Web Server + API|
|:---|:---------|:-----------|
|Review | HTML structure (tags, attributes, etc.) | Parameters and structure from documentation|


### Hacker News Example
[Hacker News](https://news.ycombinator.com/) is a social 
- It also offers an API providing structured, JSON-formatted results
    - Base URL: https://hacker-news.firebaseio.com/v0
 
- See explanation and documentation at: http://github.com/HackerNews/API

- The new Python 


In [ ]:
# source: adpated from Broucke & Baessen (Chp. b9)
import requests 
from bs4 import BeautifulSoup

# articles is an list that will hold info about each article 
articles = []

url = 'http://news.ycombinator.com/news'
r = requests.get(url)
html_soup = BeautifulSoup(r.text, 'html.parser')

for item in html_soup.find_all('tr', class = 'athing'):
    item_a = item.find('a', class_ = 'storylink')
    item_link = item_a.get('href') if item_a else None
    next_row = item.find_next_sibling('tr')
    item_score = next_row.find('span', class = 'score')
    item_score = item_score.get_text(strip = True) if item_score else '0 points'
# append the article info 


In [15]:
import requests 
articles = []
url = 'https://hacker-news.firebaseio.com/v0'
top_stories = requests.get(url + '/topstories.json').json()

print(len(top_stories))
type(top_stories)

500


list

In [ ]:
for story_id in top_stories:
    story_url = url + '/item/{}.json'.format(story_id)
    print("Fetching:", story_url)
    r = requests.get(story_url)
    story_dict = r.json()
    articles.append(story_dict)
    

### How to retrieve the top 10 stories 
First method: query encoding in API requests

In [21]:
url10 = 'https://hacker-news.firebaseio.com/v0/topstories.json?limitToFirst=10&orderBy="$key"'
ten_top_stories = requests.get(url10).json()
print(ten_top_stories)

[29110444, 29108648, 29106824, 29110419, 29109092, 29109492, 29110673, 29104047, 29110514, 29109655]


The other approach consists of defining a dict and pass it as a parameter
   - this along with the headers allows to make a more specific request to an API
   - it's recommended when developer key is needed 
   
### Specific API requests
 
- To make the API requests more specific, use headers and parameters in the request
     - Headers
     - Parameters are like filters to modify the scope of the request
         - check API documentation
         
- Reddit Example
     - With a user-agent header 

In [37]:
# Scrap the news in Reddit
# Code is adapted from the website: 
# http://towardsdatascience.com/a-beginners-guide-to-accessing-data-withweb-apis-using-python-23d262181467
import requests, json

payload = {
    'limit': 5,
    't': 'hot'
}

headers = {
    'User-agent': 'Reddit bot 1.0'
}

endpoint = 'http://www.reddit.com/r/news/top.json'
# can try other channel
# endpoint = 'http://www.reddit.com/r/funny/top.json'
endpoint = 'http://www.reddit.com/r/Baruch/new.json'

r = requests.get(endpoint, headers = headers, params = payload)
r = json.loads(r.content)

# import pprint
# pprint.pprint(r)

for sub in (r['data']['children']):
    title = sub['data']['title']
    print(title)



ECO 1001 Professor Thoughts: Andrew Green, Ernestro Garica, or Somayeh Ahmadi?
What does staff mean under instructor?
Mitchell Cohen for POL 1101?
Opinions and thoughts of the class
Switching to Operations Management


### Authenticatoin for News API

News API is a simple HTTP REST API for searching and retrieving live articles from various sources

- Get your secret API key
    - Go to: https://newsapi.org/docs/get-started

- MY API key: [*****************************************](e66483a4f4b0480a9a29a4fac0586469)

- Read the terms of service: https://newsapi.org/terms
    - attribution: The attribution should preferrably be a hyperlink to https://newsapi.org with the text "Powered by News API".

In [33]:
from newsapi import NewsApiClient
# Init
newsapi = NewsApiClient(api_key='e66483a4f4b0480a9a29a4fac0586469')

# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(q='bitcoin',
                                          sources='bbc-news,the-verge',
                                          category='business',
                                          language='en',
                                          country='us')
# /v2/everything
all_articles = newsapi.get_everything(q='bitcoin',
                                      sources='bbc-news,the-verge',
                                      domains='bbc.co.uk,techcrunch.com',
                                      from_param='2017-12-01',
                                      to='2017-12-12',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)

# /v2/top-headlines/sources

sources = newsapi.get_sources()

ModuleNotFoundError: No module named 'newsapi'

In [47]:
# Ex.4
# source: https://www.geeksforgeeks.org/fetching-top-news-using-news-api/
# BBC news api with authorization header and parameters

import requests 

# headers to store the API key
headers = {'Authorization': 'e66483a4f4b0480a9a29a4fac0586469'}
query_params = {
      "source": "bbc-news",
      "sortBy": "top"
}
main_url = " https://newsapi.org/v1/articles"
 
# fetching data in json format
res = requests.get(main_url, headers = headers, params=query_params)
open_bbc_page = res.json()
 
# getting all articles in a string article
article = open_bbc_page["articles"]
 
# empty list to hold all trending news
results = []
     
for ar in article:
    results.append(ar["title"])

# printing all trending news       
for i in range(len(results)):            
    print(i + 1, results[i], len(results[i]))


1 Covid: Record German cases as WHO warns of Europe deaths 56
2 Climate change: Facebook fails to flag denial, study finds 58
3 Ahmaud Arbery: Nearly all-white jury chosen in black jogger murder trial 72
4 How a medieval English law affects the US gun control debate 60
5 LA 'jetpack man' was probably a balloon 39
6 'I hope people would be more sceptical today' - Knox 52
7 First pill to treat Covid gets approval in UK 45
8 High-risk Covid gene more common in South Asians 48
9 Trump-Russia Steele dossier analyst charged with lying to FBI 61
10 COP26: Indonesia criticises 'unfair' deal to end deforestation 62


In [43]:
help(requests)

Help on package requests:

NAME
    requests

DESCRIPTION
    Requests HTTP Library
    ~~~~~~~~~~~~~~~~~~~~~
    
    Requests is an HTTP library, written in Python, for human beings.
    Basic GET usage:
    
       >>> import requests
       >>> r = requests.get('https://www.python.org')
       >>> r.status_code
       200
       >>> b'Python is a programming language' in r.content
       True
    
    ... or POST:
    
       >>> payload = dict(key1='value1', key2='value2')
       >>> r = requests.post('https://httpbin.org/post', data=payload)
       >>> print(r.text)
       {
         ...
         "form": {
           "key1": "value1",
           "key2": "value2"
         },
         ...
       }
    
    The other HTTP methods are supported - see `requests.api`. Full documentation
    is at <https://requests.readthedocs.io>.
    
    :copyright: (c) 2017 by Kenneth Reitz.
    :license: Apache 2.0, see LICENSE for more details.

PACKAGE CONTENTS
    __version__
    _internal_utils